In [10]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2023-02-26 23:00:09,324: Credentials are already in use. The existing account in the session will be replaced.


In [11]:
import numpy as np
from qiskit import QuantumCircuit, Aer
from qiskit.circuit import Parameter
from qiskit.opflow import StateFn, PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms import VQE


In [12]:
# Define quantum layer
def quantum_layer(x):
    circuit = QuantumCircuit(2)
    zz_map = ZZFeatureMap(2, reps=1)
    zz_map.compose(circuit, inplace=True)
    circuit.barrier()
    theta = Parameter('theta')
    circuit.rx(theta, range(2))
    circuit.barrier()
    circuit.cx(0, 1)
    return circuit

# Define PQC
def pqc(theta):
    circuit = QuantumCircuit(2)
    circuit.compose(quantum_layer(0), inplace=True)
    circuit.barrier()
    circuit = circuit.bind_parameters({theta_i: theta[i] for i, theta_i in enumerate(theta)})
    return circuit


In [13]:
# Define classical layer
def classical_layer(x, weights):
    layer1 = np.dot(x, weights[0]) + weights[1]
    layer1_activation = np.maximum(layer1, 0)
    layer2 = np.dot(layer1_activation, weights[2]) + weights[3]
    return layer2


In [14]:
# Perform data encoding
bond_length = 1.4

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

circuit = pqc([bond_length]*3)
psi = StateFn(circuit, is_measurement=True)
expect_op = PauliSumOp.from_list([('Z'*2, 1)])
expectation = (~psi @ expect_op @ psi).eval().real


Traceback (most recent call last):
  Input In [14] in <cell line: 7>
    circuit = pqc([bond_length]*3)
  Input In [12] in pqc
    circuit.compose(quantum_layer(0), inplace=True)
  Input In [12] in quantum_layer
    zz_map = ZZFeatureMap(2, reps=1)
NameError: name 'ZZFeatureMap' is not defined

Use %tb to get the full traceback.


In [ ]:
# Define loss function
def loss(theta):
    circuit = pqc(theta)
    psi = StateFn(circuit, is_measurement=True)
    expect_op = PauliSumOp.from_list([('Z'*2, 1)])
    expectation = (~psi @ expect_op @ psi).eval().real
    return expectation

# Optimize parameters using classical optimization techniques
optimizer = SPSA(maxiter=100)
vqe = VQE(expect_op, pqc, optimizer=optimizer, quantum_instance=quantum_instance)
vqe_result = vqe.compute_minimum_eigenvalue()
print(vqe_result.eigenvalue.real)


In [ ]:
!pip install pyscf
from pyscf import gto, scf
mol = gto.M(atom='Be 0 0 0; H 0 0 1.4', basis='sto3g')
mf = scf.RHF(mol)
mf_energy = mf.kernel()
print("HF energy of BeH2 molecule: ", mf_energy)


In [ ]:
!pip install qiskit_nature
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.transformers import ActiveSpaceTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

# Define electronic structure problem
driver = PySCFDriver(mol)
escf = driver.run()
problem = ElectronicStructureProblem(driver)

# Define active space transformer
num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
remove_list = [i for i in range(num_spin_orbitals) if i not in range(4)]
tr1 = ActiveSpaceTransformer(num_particles=num_particles, num_spin_orbitals=num_spin_orbitals, remove_orbitals=remove_list)
problem = tr1.transform(problem)

# Convert to qubits
converter = QubitConverter(mapper_type='JordanWigner')
qubit_op = converter.convert(problem.second_q_ops())


In [ ]:
# Define VQE algorithm
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

optimizer = SPSA(maxiter=100)
vqe = VQE(qubit_op, pqc, optimizer=optimizer, quantum_instance=quantum_instance)
vqe_result = vqe.compute_minimum_eigenvalue()
print("Ground state energy of BeH2 molecule: ", vqe_result.eigenvalue.real + mf_energy)


In [ ]:
from qiskit.circuit.library import ZZFeatureMap

num_qubits = 4
reps = 2
encoding_circuit = ZZFeatureMap(num_qubits, reps)


In [ ]:
from qiskit.circuit.library import RealAmplitudes

pqc = RealAmplitudes(num_qubits, reps)


In [ ]:
from qiskit.opflow import PauliSumOp, Z, I
from qiskit.circuit import Parameter

# Define the operator to measure
op = PauliSumOp.from_list([("Z" * num_qubits, 1.0)])
coeff = Parameter('coeff')
meas_op = coeff * op

# Connect encoding circuit and PQC with measurements
qubits = list(range(num_qubits))
meas_circuit = encoding_circuit.bind_parameters({p: [qubits[i], qubits[i+1]] for i in range(num_qubits-1)})
meas_circuit.measure_all()
meas_pqc = pqc.bind_parameters({p: [qubits[i], qubits[i+1]] for i in range(num_qubits-1)})
meas_pqc.measure_all()

# Combine the encoding circuit, PQC, and measurement circuit
qc = encoding_circuit.combine(meas_pqc).combine(meas_circuit)


In [ ]:
from qiskit.circuit.library import TwoLayerPerceptron

classical_nn = TwoLayerPerceptron(num_input_qubits=num_qubits, num_hidden_neurons=2, num_output_qubits=1, activation='relu')


In [ ]:
from qiskit.opflow import StateFn
from qiskit.algorithms.optimizers import GradientDescent

# Define the cost function
def cost_function(params):
    # Update the parameters in the quantum circuit
    qc_params = {p:


In [ ]:
# Define the molecular Hamiltonian for BeH2 molecule
molecule = Molecule(geometry=[('Be', (0, 0, 0)), ('H', (0, 0, 1.6)), ('H', (0, 0, -1.6))], charge=0, multiplicity=1)
hamiltonian = molecule.get_molecular_hamiltonian()

# Define the VQE algorithm
backend = Aer.get_backend('statevector_simulator')
optimizer = COBYLA(maxiter=1000)
vqe = VQE(ansatz=pqc, optimizer=optimizer, quantum_instance=backend, operator=hamiltonian)

# Run the VQE algorithm to estimate the ground-state energy
result = vqe.compute_minimum_eigenvalue()

# Define the loss function
true_energy = result.eigenvalue.real
def loss(params):
    energy = vqe.get_operator().assign_parameters(params).compute_eigenvalues()[0].real
    return (energy - true_energy)**2


In [ ]:
# Define the initial set of parameters
init_params = np.random.rand(pqc.num_parameters)

# Optimize the parameters using the L-BFGS-B optimizer
optimizer = L_BFGS_B(maxfun=1000)
result = optimizer.optimize(num_vars=pqc.num_parameters, objective_function=loss, initial_point=init_params)

# Get the optimized parameters and the optimized energy
opt_params = result[0]
opt_energy = vqe.get_operator().assign_parameters(opt_params).compute_eigenvalues()[0].real

# Print the optimized parameters and energy
print("Optimized parameters:", opt_params)
print("Optimized energy:", opt_energy)


In [ ]:
# Define the Hamiltonian operator for the molecule
hamiltonian = MolecularGroundStateEnergy(molecule_transformer, qubit_converter)

# Assign the optimized parameters to the PQC
pqc.assign_parameters(opt_params)

# Create an instance of the StateFn class for the Hamiltonian operator
op = hamiltonian.to_op()

# Compute the expectation value of the Hamiltonian operator for the given quantum state
expectation_value = op.assign_parameters(pqc.parameters).eval()
ground_state_energy = expectation_value.real

# Print the ground-state energy
print("Ground-state energy:", ground_state_energy)


In [ ]:
# Define the bond length operator for the molecule
bond_length = MolecularBondLength(molecule_transformer, qubit_converter)

# Create an instance of the StateFn class for the bond length operator
op = bond_length.to_op()

# Compute the expectation value of the bond length operator for the given quantum state
expectation_value = op.assign_parameters(pqc.parameters).eval()
bond_length = expectation_value.real

# Print the bond length
print("Bond length:", bond_length)


In [16]:
# Define the vibrational frequency operator for the molecule
vibrational_frequency = MolecularVibrationalFrequency(molecule_transformer, qubit_converter)

# Create an instance of the StateFn class for the vibrational frequency operator
op = vibrational_frequency.to_op()

# Compute the expectation value of the vibrational frequency operator for the given quantum state
expectation_value = op.assign_parameters(pqc.parameters).eval()
vibrational_frequency = expectation_value.real

# Print the vibrational frequency
print("Vibrational frequency:", vibrational_frequency)


Traceback (most recent call last):
  Input In [16] in <cell line: 2>
    vibrational_frequency = MolecularVibrationalFrequency(molecule_transformer, qubit_converter)
NameError: name 'MolecularVibrationalFrequency' is not defined

Use %tb to get the full traceback.


In [15]:
print("Ground state energy: ", result.fun)


Traceback (most recent call last):
  Input In [15] in <cell line: 1>
    print("Ground state energy: ", result.fun)
NameError: name 'result' is not defined

Use %tb to get the full traceback.
